In [ ]:
# Import the neccesary python libraries for Bayesian analysis
# and data visualisation.
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import stan as ps
import arviz as az
import statistics as Stats
from patsy import dmatrix
import os

# Importing nest_asyncio is only necessary to run pystan in Jupyter Notebooks.
import nest_asyncio
nest_asyncio.apply();

# Universal variables
pp_samples = 100
# Specify the number of chains to the number of availible cpu's.
n_chains = os.cpu_count()
n_samples = 2000
#Convert to int so Stan will not crash below.
n_warmup = int(n_samples/2)
#Specify step size.
stepS = .8

In [ ]:
from IPython.core.display import HTML as Center

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

# Bayesian estimation of skew normal variable

The following notebook demonstrates a very useful but simple example of an application of Bayesian data analysis. That application being that if their is a parameterised probability distribution and you have enough data fitting a statistical model to that data based on that probability distribution in the form of a likelihood is feasible and therefore, so is statistical inference. Remember Bayesian data analysis is simply the application of Bayes rule, $P(\theta \mid y) = \large \frac{P(y \mid \theta) \, P(\theta)}{P(y)}$ to statistical models Gelman et al. (2013).

This may not be a surprise to any researchers who understand and apply maximum likelihood methods. However, that is a skillset not possessed and arguably not even within the purview of many psychological researchers who are taught statistical tests based or gaussian assumptions (Delacre, Lakens, & Leys, 2017, McElreath, 2020) and thus dominates their statistical practices. A primary advantage for researchers which this notebook hopes to show in a simple form is the use Bayesian data analysis to build models that describe the data generating process for the phenomenon under study (Kruschke, 2015; McElreath, 2020).

Therefore, the example below demonstrates the fitting of a simple skew-normal model to some multiple observations of a single subjects reaction time data to a stroop task. In this example many researcher would apply a Null hypothesis significance test (NHST) likely in the form of one sample t-test or potentially even non-parametric test when the data is skewed like the data here (Lachaud & Renaud, 2011) with various justifications. The exact issues with this are diverse and therefore not explained in any further detail here.

# Steps of Bayesian data analysis

<font size = "3"> Kruschke (2015) offers a step by step formulation for how to conduct a Bayesian analysis:

1. Identify the relevant data for the question under investigation.

2. Define the descriptive (mathematical) model for the data.

3. Specify the Priors for the model. If scientific research publication is the goal the priors must be accepted by a skeptical audience. Much of this can be achieved using prior predictive checks to ascertain if the priors are reasonable.

4. Using Bayes' rule estimate the posterior for the parameters of the model using the likelihood and priors. Then interpret the posterior.

5. Conduct model checks. i.e. Posterior predictive checks.</font> 

<font size = "1">This notebook will follow this approach generally.</font> 

#  Step 1 - Identify the relevant data for  the question under investigation

The data that is analysed below actually comes from another Bayesian teaching resource https://vasishth.github.io/bayescogsci/book/ex-compbda.html. It is actually suggested as an end of chapter exercise. The author of this notebook openly admits they were struggling to find a nice example for a skew normal model to show a not so common application of Bayesian inference (with out simple simulatinf) and thanks the authors for the inspiration. The authors of that textbook as far as the author of this notebook is aware advocate for open science/education and will not have an issues with the use of their data. Of course, if their are any issues please make the author of this notebook aware. Anyhow, the data is imported from the associated github page for online textbook linked above.

In [ ]:
# Url variable for the raw data for the reaction times of the single participant.
url = 'https://raw.githubusercontent.com/bnicenboim/bcogsci/master/data-raw/datasets/button_press.csv'

In [ ]:
# Generate pandas dataframe for the reaction time dataset
df = pd.read_csv(url)

In [ ]:
# Output the headers of the dataset.
df.head(1)

## Exploratory data analysis

In [ ]:
# Plot the reaction time data for the single participants repeated trial observations.
sns.displot(df['rt']).set(title = 'Single particpants Stroop task trial reaction times scores', 
                          xlabel = 'Reaction time');

# Step 2 - Define the descriptive statistical model \begin{align*}
y_{i} &\sim SkewNormal(\xi, \omega, \alpha)
\\ \xi &\sim Normal(\bar{y}, \sigma_y * 5)
\\ \omega &\sim Normal(0, \sigma_y * 10)
\\ \alpha &\sim Normal(0, 10)
\end{align*} 


### Step 3 - Specifying priors
The prior values here were selected to be broad and data informed in line with the suggestions of prior setting in Kruschke (2015). This is not such a problem here as there is large number of data points and the models are simple (low-dimensional).

## Stan model for skew-normal 

In [ ]:
skew_normal = '''

// Functions defined here are simple but help demonstrate 
// and clean up the generated quantities block
// which is generaly appropriate with writing
// Clearer Stan code. 

functions{
// Function to calculate the expected value of skew normal random variable. 
real skew_normal_mean(real xi, real omega, real alpha){
    real delta = alpha / sqrt(1 + pow(alpha, 2));
    real mean_sn = xi + omega * delta * sqrt(2/pi());
    return mean_sn;
    }
}

data{

int<lower = 1> N; // Set total observations
vector[N] y; // vector of dependent variable values

// Variable to specify if prior or posterior predictive checks are run.
int<lower = 0, upper = 1> onlyprior;

// Prior parameter values
real xi_mu;
real xi_sigma;
real omega_sigma;
real alpha_sigma;
}
parameters{

// Parameters for Stan parameterisation of skew normal distribution
real xi;
real<lower = 0> omega;
real alpha;

}

model{
// priors
xi ~ normal(xi_mu,xi_sigma);
omega ~ normal(0, omega_sigma);
// This prior represents that the skew could be in either left/right.
alpha ~ normal(0, alpha_sigma);

// Run likelihoood if onlyprior is set to 0
if(!onlyprior){
    y ~ skew_normal(xi, omega, alpha);
    }
}

generated quantities{
vector[N] yrep;
vector[N] log_lik;
real sn_mean;

sn_mean = skew_normal_mean(xi, omega, alpha);

// Need for loop as skew_normal_rng is not vectorisable. See Stan manual for updates
for(i in 1:N){
    yrep[i] = skew_normal_rng(xi, omega, alpha);
    }

// Only run log likelihood when posterior is estimated    
if(!onlyprior){
for (i in 1:N) {
    log_lik[i] = skew_normal_lpdf(y[i] | xi, omega, alpha);
      }
  }  
}
'''

In [ ]:
# Generate python dictionary to be passed to stan during build of the model for later sampling.
data_sn = {'N': len(df),
        'y': df['rt'].values,
        #Set as 1 fro prior predictive checks
        'onlyprior': 0,
        'xi_mu': np.mean(df['rt']),
        'xi_sigma': np.std(df['rt']) * 5,
        'omega_sigma': np.std(df['rt']) * 10,
        'alpha_sigma': 10.,
           }

In [ ]:
# Compile stan model tp C++ code
sm = ps.build(skew_normal, data = data_sn)

In [ ]:
# Sample from the Stan model.
fit = sm.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS);

In [ ]:
# Geneate panda data frame of the posterior samples with metrics calculated in addition.
fit_dfsn = fit.to_frame()

# Calculate the number of divergences
np.sum(fit_dfsn['divergent__'])

In [ ]:
az.summary(fit, var_names = ['xi', 'omega', 'alpha', 'sn_mean'])

In [ ]:
az.plot_posterior(fit, var_names = ['xi', 'omega', 'alpha', 'sn_mean']);

In [ ]:
# Using the arviz package the traceplots of the 4 MCMC chains can be plotted.
az.plot_trace(fit, var_names = ("xi", "omega", "alpha"));

In [ ]:
# Using the arviz package generate rank plot for the MCMC chains to assess mixing.
az.plot_rank(fit, var_names = ("xi", "omega", "alpha"));

The rank plots suggest little deviance from uniforminity of the chains. Suggesting that they have mixed well.

In [ ]:
# Convert pystan fit object to IO for Arviz functions.
idata = az.from_pystan(posterior = fit, posterior_model = sm, log_likelihood = ['log_lik'], posterior_predictive = ['yrep'], observed_data= 'y')

In [ ]:
# Plot posterior simulated data sets for posterior predictive check
if data_sn['onlyprior'] == 0:
    az.plot_ppc( idata, data_pairs= {"y" : "yrep"}, num_pp_samples = pp_samples, observed = True );
else:
    az.plot_ppc( idata, data_pairs= {"y" : "yrep"}, num_pp_samples= pp_samples, observed = False );

Posterior predcitive plots show that the skew normal model capture the participants reaction times fairly well. However, it fails to capture the outlier reaction time outlier score. This is a result of the skew-normal unsurprisingly still have the skinny tail of the regular normal distribution.

# Can we do better?

## Lognormal model for reaction times


# Define the descriptive statistical model \begin{align*}
y_{i} &\sim LogNormal(\mu, \sigma)
\\ \mu &\sim Normal(6, 1.5)
\\ \sigma &\sim Normal(0, 1)
\end{align*} 

The prior values here were selected to be consistent with the ones used in https://vasishth.github.io/bayescogsci/book/ex-compbda.html. The amount of data here again makes the priors less of a concern in this example.

### Stan model for lognormal

In [ ]:
ln_model = '''
// Functions defined here are simple but help demonstrate 
// and clean up the generated quantities block
// which is generaly appropriate with stan code. 

functions{
real logn_median(real mu){
    return exp(mu);
}
real logn_mean(real mu , real sigma){
    return exp(mu + pow(sigma, 2) / 2);
    }
}

data{

int<lower = 1> N; // Set total observations
vector[N] y; // vector of dependent variable values

// Variable to specify if prior or posterior predictive checks are run.
int<lower = 0, upper = 1> onlyprior;

// Prior parameter values
real mu_mu;
real mu_sigma;
real sigma_sigma;
}
parameters{

// Parameters for Stan parameterisation of lognormal distribution
real mu;
real<lower = 0> sigma;

}

model{

// Priors
mu ~ normal(mu_mu, mu_sigma);
sigma ~ normal(0, sigma_sigma);

// Run likelihoood if onlyprior is set to 0
if(!onlyprior){
    y ~ lognormal(mu, sigma);
    }
}

generated quantities{
vector[N] yrep;
vector[N] log_lik;

// Calculate the posterior for the median value of a lognormal random variable.
real median_ln = logn_median(mu);

// Calculate the posterior for the mean value of a lognormal random variable.
real mean_ln = logn_mean(mu , sigma);

// Need for loop as skew_normal_rng is not vectorisable. see Stan manuals for updates
for(i in 1:N){
    yrep[i] = lognormal_rng(mu, sigma);
    }
    
// Only run log likelihood when posterior is estimated    
if(!onlyprior){
for (i in 1:N) {
    log_lik[i] = lognormal_lpdf(y[i] | mu, sigma);
      }
  }
}
'''

In [ ]:
data_ln = {'N': len(df),
           'y': df['rt'].values,
           # Set to 0 to run likelihood and fit the model 
           'onlyprior': 0,
           'mu_mu': 6.,
           'mu_sigma': 1.5,
           'sigma_sigma': 1
          }

In [ ]:
# Compile the lognormal Stan model to C++.
sm_ln = ps.build(ln_model, data = data_ln)

In [ ]:
# Smaple the psoterios for the lognormal model.
fit_ln = sm_ln.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS)

In [ ]:
# Output summary of model parameters
az.summary(fit_ln, var_names = ['mu', 'sigma', 'median_ln', 'mean_ln'])

In [ ]:
# Generate pandas dataframe with posterior samples and key metrics calculated.
fit_lndf = fit_ln.to_frame()

In [ ]:
# Calculate the number of divergences
np.sum(fit_lndf['divergent__'])

## Expected value of lognormal variable posteriors

In [ ]:
az.plot_posterior(fit_ln, var_names = ['median_ln', 'mean_ln']);

In [ ]:
az.plot_trace(fit_ln, var_names = ['mu', 'sigma']);

In [ ]:
az.plot_rank(fit_ln, var_names = ("mu", "sigma"));

In [ ]:
idata_ln = az.from_pystan(posterior = fit_ln, posterior_model = sm_ln, log_likelihood = ['log_lik'],
                          posterior_predictive=['yrep'],observed_data= 'y')

In [ ]:
if data_ln['onlyprior'] == 0:
    az.plot_ppc( idata_ln, data_pairs= {"y" : "yrep"}, num_pp_samples= pp_samples, observed = True );
else:
    az.plot_ppc( idata_ln, data_pairs= {"y" : "yrep"}, num_pp_samples= pp_samples, observed = False );

### How does a normal distribution model fair?

To coincide with what most psychological researchers are used to applying it is useful to see how a normal distributional model fairs when modelling this data set of reaction times.

## Define the descriptive statistical model \begin{align*}
y_{i} &\sim Normal(\mu, \sigma)
\\ \mu &\sim Normal(\bar{y}, \sigma_y * 5)
\\ \sigma &\sim Normal(0, \sigma_y * 10)
\end{align*} 

The prior values here were selected to coincide with the skew normal model as close as possible. Of course, the skewness parameter ($\alpha$) makes a large difference to the model skew-normal model

## Stan model for Normal

In [ ]:
n_model = '''
data{

int<lower = 1> N; // Set total observations
vector[N] y; // vector of dependent variable values

// Variable to specify if prior or posterior predictive checks are run.
int<lower = 0, upper = 1> onlyprior;

// Prior parameter values
real mu_mu;
real mu_sigma;
real sigma_sigma;
}
parameters{

// Parameters for Stan parameterisation of lognormal distribution
real mu;
real<lower = 0> sigma;

}

model{
// Priors
mu ~ normal(mu_mu, mu_sigma);
sigma ~ normal(0, sigma_sigma);

// Run likelihoood if onlyprior is set to 0
if(!onlyprior){
    y ~ normal(mu, sigma);
    }
}

generated quantities{
vector[N] yrep;

vector[N] log_lik;
for (i in 1:N) {
    log_lik[i] = normal_lpdf(y[i] | mu, sigma);
      }

for(i in 1:N){
    yrep[i] = normal_rng(mu, sigma);
    }

}
'''

In [ ]:
data_n = {'N': len(df),
           'y': df['rt'].values,
           'onlyprior': 0,
           'mu_mu': np.mean(df['rt']),
           'mu_sigma': np.std(df['rt']) * 5,
           'sigma_sigma': np.std(df['rt']) * 10,
          }

In [ ]:
sm_n = ps.build(n_model, data = data_n)

In [ ]:
fit_n = sm_n.sample(num_chains = n_chains , num_samples = n_samples, num_warmup = n_warmup, stepsize = stepS)

In [ ]:
az.summary(fit_n, var_names = ['mu', 'sigma'])

# Traceplots normal model

In [ ]:
az.plot_posterior(fit_n, var_names = ['mu', 'sigma']);

# Traceplots normal model

In [ ]:
az.plot_trace(fit_n, var_names = ['mu', 'sigma']);

In [ ]:
az.plot_rank(fit_n, var_names = ("mu", "sigma"));

In [ ]:
# Convert pystan fit object to IO for Arviz functions.
# Convert pystan fit object to IO for Arviz functions.
idata_n = az.from_pystan(posterior=fit_n, posterior_model = sm_n, log_likelihood = ['log_lik'], 
                       posterior_predictive=['yrep'],observed_data= 'y')

In [ ]:
if data_n['onlyprior'] == 0:
    az.plot_ppc( idata_n, data_pairs= {"y" : "yrep"}, num_pp_samples= pp_samples, observed = True );
else:
    az.plot_ppc( idata_n, data_pairs= {"y" : "yrep"}, num_pp_samples= pp_samples, observed = False );

In [ ]:
# Generate python dictionary to label model with the correct data.
compare_dict = {"Model_SN": idata, "Model_LN": idata_ln, "Model_N": idata_n}

# Compare models using psis leave one out cross validated.
# Output model comparison results.
model_compare = az.compare(compare_dict, scale = 'deviance');
print(model_compare)
az.plot_compare(model_compare);

In [ ]:
expectations = [fit["sn_mean"], fit_ln["mean_ln"], fit_n["mu"] ]
expectation_labs = ["Model_SN", "Model_LN" , "Model_N"]
for i in range(0,len(expectations)):
    az.plot_posterior(expectations[i])
    plt.title(expectation_labs[i])

The model comparison results above show some interesting things. Firstly, the performance of the models in terms of out of sample prediction are not surprsing seeing as they are attempting to model reaction time data with the lognormal model performing the best. However, as the model comparison plot shows that this is only in terms of point estimation as there is a lot of overlap in terms of the estimated error and that uncertainty increases as you go from LN-SN-N model. Another interesting outcome to look at with the model comparison results is that SN and N model both resulted in warnings for highly leveraging points where as LN did not. Which suggests that LN model was more robust to the outlier value. What is also of interest in that the estiamtes of each model in terms of expectations were very similar

This notebook has hopefully shown that Bayesian data analysis offers robust and flexible modelling options. In addition, it should hopefully show that even in this simple situation more appropriate modelling options exist and that there are alternatives to the general dominance of gaussian assumptions within standard statistical tests. It has to be said though that application of these alternatives is often not as simple as gaussian models. In that the gaussian distribution is nice, in terms that it is easier to fit to data (computationally) and requires less post work to understand. However, we can likely do better in situations that call for these alternatives. Additionally, we do not have to commit to a single model and the studying multiple models is likely to be fruitful in understanding psychological phenomena (Smaldino, 2017).

# References

Delacre, M., Lakens, D., & Leys, C. (2017). Why psychologists should by default use Welch’s t-test instead of Student’s t-test. International Review of Social Psychology, 30(1).

Kruschke, J. K., (2015). Doing bayesian data analysis: A tutorial with R, JAGS, and Stan. New york, NY: Academic Press.

Lachaud, C. M., & Renaud, O. (2011). A tutorial for analyzing human reaction times: How to filter data, manage missing values, and choose a statistical model. Applied Psycholinguistics, 32(2), 389-416.

McElreath, R. (2020). Statistical rethinking: A Bayesian course with examples in R and Stan. Boca Raton: CRC Press.

Smaldino, P. E. (2017). Models are stupid, and we need more of them. Computational social psychology, 311-331.

## Exercises for the motivated readers

1. Change the prior parameter values and onlyprior variable (set to 1) in the python dictionaries for the models and see the effects on prior predictive distributions.

2. Fit a student-t distribution model and do model comparison to see the outcomes.

3. Use tighter priors on the models and observe the outcomes for posterior predictive checks and LOO estimates.